In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.metrics import confusion_matrix, classification_report

# Configuración inicial
image_size = (128, 128)  # Tamaño de las imágenes para redimensionar
batch_size = 32

# Ruta al dataset (ajusta esto a tu entorno)
dataset_path = "./COVID-19_Radiography_Dataset"

# Subcarpetas de las clases
categories = ["COVID", "NORMAL", "PNEUMONIA", "Lung_Opacity"]

# Configurar generador de datos con aumento
datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=5,  # Reduce rotación
    width_shift_range=0.05,  # Reduce desplazamiento
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True
)


# Crear generadores para entrenamiento y validación
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Cargar el modelo MobileNetV2 preentrenado sin las capas superiores
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Congelar las capas de la base
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Congela la mayoría, pero descongela las últimas 50 capas
    layer.trainable = False


# Construir el modelo
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation='softmax')
])

# Compilación del modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Configurar Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Entrenamiento
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluar en el conjunto de validación
test_loss, test_acc = model.evaluate(val_generator, verbose=2)
print(f"Exactitud en validación: {test_acc:.2f}")

# Graficar curvas de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Exactitud entrenamiento')
plt.plot(history.history['val_accuracy'], label='Exactitud validación')
plt.xlabel('Épocas')
plt.ylabel('Exactitud')
plt.legend()
plt.title('Curvas de entrenamiento y validación')
plt.show()

# Predicciones
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = val_generator.classes

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred_classes)
print("\nMatriz de confusión:\n", cm)

# Informe de clasificación
print("\nInforme de clasificación:\n", classification_report(y_true, y_pred_classes, target_names=categories))


Found 33866 images belonging to 4 classes.
Found 8464 images belonging to 4 classes.
Epoch 1/20
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 573s 531ms/step - accuracy: 0.6817 - loss: 0.8502 - val_accuracy: 0.7413 - val_loss: 1.3015
Epoch 2/20
 368/1059 ━━━━━━━━━━━━━━━━━━━━ 5:35 486ms/step - accuracy: 0.7654 - loss: 0.6234